In [2]:
import  urllib3
import facebook
import requests
import pandas as pd
from datetime import date, timedelta
from os import listdir
import numpy as np
from tqdm import tqdm
import time
from time import sleep
import math

## Functions

In [37]:
def cnx_graph_api(TOKEN):
    #with open('graph_api_authentication.txt') as file:
    #    TOKEN = file.readline().split('"')[1]
    TOKEN = TOKEN
    graph = facebook.GraphAPI(access_token=TOKEN, version = 3.1)
    return graph
def get_engagement_link(link, id_factcheck, TOKEN):

    graph = cnx_graph_api(TOKEN)
    engagement = graph.request(link +'?fields=engagement')
    if len(engagement) == 1:
        data = []
        df = pd.DataFrame(data, columns=['url','id_factcheck','reaction_count', 'comment_count', 'share_count', 'comment_plugin_count'])
        new_row = {'url': link,'id_factcheck':id_factcheck,'reaction_count': np.nan, 'comment_count':np.nan, 'share_count': np.nan, 'comment_plugin_count': np.nan,}
        df = df.append(new_row, ignore_index=True)
    else:
        df = pd.DataFrame([engagement['id']], columns = ['url'])
        df['id_factcheck'] = id_factcheck
        df = pd.concat([df.reset_index(drop=True), pd.DataFrame([engagement['engagement']])], axis=1)

    return (df)
def get_engagement_all_links(links, id_articles):

    data = []
    df_all = pd.DataFrame(data, columns=['url', 'id_factcheck', 'reaction_count', 'comment_count', 'share_count', 'comment_plugin_count'])
    for link, id_ in zip(links, id_articles):
        df = get_engagement_link(link, id_)
        df_all = df_all.append(df, ignore_index=True)

    return (df_all)

In [10]:
def load_factchecks():
    factchecks = pd.read_excel('/Users/cblanesg/misinformation_socialmedia/data/1-factchecks/2-clean_factchecks/factchecks_engagements.xlsx').drop('Unnamed: 0', axis = 1)
    return(factchecks)

## Apply Functions

In [11]:
factchecks = load_factchecks()

In [12]:
df_dict = {n: factchecks.iloc[n:n+240, :]
           for n in range(0, len(factchecks), 240)}

In [32]:
tokens = ['EAAJ9JaxDifgBABhXrpK0RXtAQAkfVEhuunLiIDNjsqiFzULZC4ZA1EHmJhGLYO4PkIhnwynOIdbbKGNePd3co0hoaYn6H8TuvIh103XH1Qcl6jLZAWlvmJ2c2uwjdMxAXa3iVXEGNmPxXdYWwaeTBnPjE6UUoFHBRZCYbcYeZB7t8SlbNX4uQXae7znGNdE2fO4Yv1NgJ7mOZBqhaZARqz770sfZBvwikuhZA4wKNTOS7ilQc8aCpzBJg', 
         'EAANMQfZAr9m0BADVVDijIhWZCWZA6uZCT30uLJVQANyVAtpVmvr0cn5ZArEFKrmhEbxiDPIOqR3iVmjWmSXcHpLKMnx9WIqpNIfu9uEZAqf78BU2QxqbyjEfi88PGPFp9cVbZCzTMGKWCyDf2uyHr5E3kU9FZCT6ZBi51UuAfzQlFakbHZBhzXL3zqfBCNiG4m7lnED0S1t1vtJQx9ZBFH3yZAi57jYcMDuOcaalk5G7wynWGAvl9FDTpL1V', 
         'EAAKRE2EGs6YBAMkAZBCdiSvqX856Yy5rZANDps1IjtjmiBt4JvxICxLXNqtHVE9IvbBK6GUdYukBzO6XeVVzjTe54C6FOU9gvCFj45vcrvTHW0xKGOZBaXhrDO2ZAJoEVIhL1hbTKvipHOzJURfCEj6IVt0H49850mFZBckt09jSQZCAKk5ysdPw8Tcq3TEb3vhCQrZAsOydbUYQEr5CtMSjweafXhwqzIWjZAcJ6WTzZCxZCQWJZAkFXMZC']

In [28]:
df_dict.keys()

dict_keys([0, 240, 480])

In [42]:
all_interactions = []

In [43]:
for x, y in tqdm(zip(list(df_dict.keys()), tokens), total = 3):
    df = df_dict[x]
    for h, w in zip(df.link_chequeo, df.id_factcheck):
        df_all = get_engagement_link(h, w, y)
        all_interactions.append(df_all)

100%|██████████| 3/3 [02:14<00:00, 44.89s/it]


In [47]:
engagement_all = pd.concat(all_interactions).drop(['comment_plugin_count', 
                                                  'url'], axis = 1)

In [49]:
engagement_all['engagements_complete'] = engagement_all.apply(lambda x: x['reaction_count'] + 
                                                             x['comment_count'] + 
                                                             x['share_count'], axis = 1)

In [52]:
factchecks_engagements = pd.merge(left = factchecks, 
         right = engagement_all, 
         on = 'id_factcheck', 
         how = 'left')

In [55]:
factchecks_engagements.engagements_complete.median()

17.0

In [60]:
factchecks_engagements['popular'] = np.where(factchecks_engagements['engagements_complete'] > factchecks_engagements.engagements_complete.median(), 1, 0)

In [62]:
factchecks_engagements.to_excel('/Users/cblanesg/misinformation_socialmedia/data/1-factchecks/2-clean_factchecks/factchecks_engagements.xlsx')

In [65]:
factchecks_engagements[['engagements_complete', 
                       'share_count', 
                       'comment_count', 
                       'reaction_count']].mean()

engagements_complete    539.263158
share_count             230.005445
comment_count            69.063521
reaction_count          240.194192
dtype: float64

In [67]:
factchecks_engagements[['engagements_complete', 
                       'share_count', 
                       'comment_count', 
                       'reaction_count']].median()

engagements_complete    17.0
share_count              8.0
comment_count            0.0
reaction_count           6.0
dtype: float64

In [77]:
factchecks_engagements[['engagements_complete', 
                       'share_count', 
                       'comment_count', 
                       'reaction_count', 
                       'popular', 'id_factcheck']].groupby(['popular']).mean()

,engagements_complete
popular,
0,2.826715
1,1081.572993
